In [3]:
import torch, json
start_id = 0
end_id = 184
path_dir = "/inspire/hdd/project/inference-chip/xujiaming-253308120313/Paper/SpecMoD/train_data/global_router/sharegpt"
avg_len = 0
for i in range(start_id, end_id):
    start_idx = i*100
    end_idx = start_idx + 100
    json_path = f"{path_dir}/sharegpt_common_en_Qwen3-8B_normal_info_{start_idx}_{end_idx}.json"
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
        for id, meta_json_data in data.items():
            avg_len += meta_json_data['avg_len']
print(avg_len / 18400)
            

28.487224634804267


In [5]:
last_hidden_states.dtype

torch.float16

In [38]:
from typing import Dict
class RouterConfig(Dict):
    def __getattr__(self, item):
        try:
            return self[item]
        except KeyError:
            raise AttributeError(f"No such attribute: {item}")

    def __setattr__(self, key, value):
        self[key] = value
    
a = {'a':2}
a = RouterConfig(a)

In [40]:
a['a']

2

In [7]:
import json
with open("/inspire/hdd/project/inference-chip/xujiaming-253308120313/Paper/SpecMoD/train_data/global_router/sharegpt/sharegpt_common_en_Qwen3-8B_normal_info_0_100.json", 'r', encoding='utf-8') as f:
    data = json.load(f)
    tot = 0
    for id, json_data in data.items():
        tot += len(json_data['output'])
tot

58434

In [8]:
data.keys()

dict_keys(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99'])

In [41]:
from transformers import AutoTokenizer,AutoModelForCausalLM
Ori_model_path = f"/inspire/hdd/global_public/public_models/Qwen/Qwen3-8B/"
tokenizer = AutoTokenizer.from_pretrained(Ori_model_path)

/inspire/hdd/global_user/xujiaming-253308120313/anaconda3/envs/specmod/lib/python3.14/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
from transformers.models.qwen3.modeling_qwen3 import Qwen3ForCausalLM
model = Qwen3ForCausalLM.from_pretrained(Ori_model_path)

/inspire/hdd/global_user/xujiaming-253308120313/anaconda3/envs/deepspeed/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NameError: name 'Ori_model_path' is not defined

In [14]:
model.model.embed_tokens

Embedding(151936, 4096)

In [35]:
torch.tensor([[2,3,4,5]]).sum().item()

14

In [3]:
last_hidden_states.shape

torch.Size([1, 23469, 12288])

In [12]:
import json
output = []
with open(f"/inspire/hdd/project/inference-chip/xujiaming-253308120313/dataset/eval/ShareGPT-Chinese-English-90k/sharegpt_jsonl/computer_en_26k(fixed).jsonl", "r", encoding="utf-8") as f:
    data = [json.loads(line) for line in f if line.strip()]
    for id, meta_data in enumerate(data):
        new_meta_data = {"question_id":id, "category":meta_data['category'], "turns":[meta_data["conversation"][0]["human"]], "reference":[meta_data["conversation"][0]["assistant"]]}
        output.append(new_meta_data)


with open("/inspire/hdd/project/inference-chip/xujiaming-253308120313/dataset/benchmark/sharegpt_computer_en/question.jsonl", "w", encoding="utf-8") as f:
    for item in output:
        # 1. json.dumps 将字典转为字符串
        # 2. ensure_ascii=False 保证中文正常显示，而不是显示成 \uXXXX
        # 3. + "\n" 手动添加换行符
        line = json.dumps(item, ensure_ascii=False)
        f.write(line + "\n")
    

In [11]:
# 用于检查数据集的每一条记录是否符合要求，不符合要求的将被删除

import json
import os

# 定义要检查的文件路径
file_path = "/inspire/hdd/project/inference-chip/xujiaming-253308120313/dataset/eval/ShareGPT-Chinese-English-90k/sharegpt_jsonl/computer_en_26k(fixed).jsonl"
# 定义用于存储有效行的列表
valid_lines = []

# 打开文件进行逐行检查
with open(file_path, "r") as file:
    for line_number, line in enumerate(file, start=1):
        line = line.strip()
        if line:
            try:
                data = json.loads(line)
                if (
                    "conversation" in data 
                    and data["conversation"] 
                    and data["conversation"][0]["human"] != ""
                    and data["conversation"][0]["assistant"] != ""
                    and data["conversation"][0]["human"] != "continue"
                ):
                    valid_lines.append(line)
                else:
                    print(f"删除第 {line_number} 行：无效的conversation")
            except json.JSONDecodeError:
                print(f"删除第 {line_number} 行：JSON解析错误")

# 删除原始文件
os.remove(file_path)

# 将有效行写入新文件
with open(file_path, "w") as file:
    for line in valid_lines:
        file.write(line + "\n")

print("检查完成并删除不符合要求的行。")

删除第 1142 行：无效的conversation
删除第 3728 行：无效的conversation
删除第 4276 行：无效的conversation
删除第 5898 行：无效的conversation
删除第 11126 行：无效的conversation
删除第 13178 行：无效的conversation
检查完成并删除不符合要求的行。


In [31]:
torch.tensor(json_data['output']).view(1,-1).shape

torch.Size([1, 199])

In [17]:
model.model.embed_tokens(torch.tensor(json_data['output'])).shape

torch.Size([199, 4096])

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM,Qwen3ForCausalLM
# /share/public/public_models/Qwen3-14B/
# /share/others/public_models/Qwen3-14B/

tokenizer = AutoTokenizer.from_pretrained("/inspire/hdd/global_public/public_models/Qwen/Qwen3-8B/")
model = AutoModelForCausalLM.from_pretrained("/inspire/hdd/global_public/public_models/Qwen/Qwen3-8B/")

import json
import torch

datasets = ['alpaca', 'gsm8k', 'sum', 'mt-bench','vicuna-bench', 'math_infini']
for dataset in datasets:
    
    with open(f"/inspire/hdd/project/inference-chip/xujiaming-253308120313/Paper/SpecMoD/train_data/router/{dataset}_Qwen3-8B_data_None_None.json", "r") as f:
        data = json.load(f)
        save_output_path = f"/inspire/hdd/project/inference-chip/xujiaming-253308120313/Paper/SpecMoD/log/{dataset}_Qwen3-8B_data_None_None.json"
        with open(save_output_path, 'w', encoding='utf-8') as f:
            for id, json_data in data.items():
                print(tokenizer.decode(json_data['output']), file=f)
# data = torch.load("/inspire/hdd/project/inference-chip/xujiaming-253308120313/Paper/SpecMoD/train_data/router/math_infini_Qwen3-8B_last_hidden_states_0_1.pt")
# print(data.shape)

In [5]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM,Qwen3ForCausalLM
tokenizer = AutoTokenizer.from_pretrained("/inspire/hdd/global_public/public_models/Qwen/Qwen3-8B/")
model = AutoModelForCausalLM.from_pretrained("/inspire/hdd/global_public/public_models/Qwen/Qwen3-8B/")

/inspire/hdd/global_user/xujiaming-253308120313/anaconda3/envs/specmod/lib/python3.14/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 5/5 [00:56<00:00, 11.30s/it]


In [6]:
inputs = tokenizer.apply_chat_template(
            "hello world",
            tokenize=True,
            return_dict=True,
            return_tensors="pt",
        )

In [6]:
a = 10
b = [1,2,3,4]
int(a in b)

0

In [22]:
a = [1,2,3]
sum(a)

6

In [21]:
import torch
1 in torch.load("/inspire/hdd/project/inference-chip/xujiaming-253308120313/Paper/SpecMoD/train_data/layer_router/alpaca_Qwen3-8B_laye_router_Y_idx3_None_None.pt")

True

In [ ]:
# 测试加载input id供微调adaptor
import json

datasets = ['alpaca', 'gsm8k', 'sum', 'mt-bench','vicuna-bench', 'math_infini']
all_finetune_data = []
for dataset in datasets:
# dataset = 'alpaca'
    with open(f"/inspire/hdd/project/inference-chip/xujiaming-253308120313/Paper/SpecMoD/train_data/adaptor/{dataset}_Qwen3-8B_data_None_None.json", "r") as f:
        data = json.load(f)
        for id, json_data in data.items():
            input_ids = json_data['Prompt']
            layer_index = []
            for token_info in json_data['Token']:
                input_ids.append(token_info['input_id'])
                layer_index.append(token_info['layer_index'])
            finetune_data = {'input_ids':input_ids, 'layer_index':layer_index}
            all_finetune_data.append(finetune_data)
save_path = f'./train_data/adaptor_finetune/finetune_data.json'
with open(save_path, "w") as f:
    json.dump(all_finetune_data, f, ensure_ascii=False, indent=4)

In [4]:
a ="hello world"
a[:a.index("world")]

'hello '

In [16]:
import torch
a = torch.ones((1, 1,100))
b = torch.ones((1, 1,100))
torch.nn.functional.cosine_similarity(a,b, dim =-1).mean()

tensor(1.)

In [14]:
import json
with open(f"/inspire/hdd/project/inference-chip/xujiaming-253308120313/Paper/SpecMoD/train_data/global_router/mt-bench_Qwen3-8B_normal_info_0_2.json", "r") as f:
    data = json.load(f)
    for id, json_data in data.items():
        print(len(json_data['output']))

190
214


In [3]:
import json
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM,Qwen3ForCausalLM
tokenizer = AutoTokenizer.from_pretrained("/inspire/hdd/global_public/public_models/Qwen/Qwen3-8B/")
model = AutoModelForCausalLM.from_pretrained("/inspire/hdd/global_public/public_models/Qwen/Qwen3-8B/")
with open('/inspire/hdd/project/inference-chip/xujiaming-253308120313/Paper/SpecMoD/train_data/adaptor_finetune/finetune_data.json', 'r') as f:
    data = json.load(f)

/inspire/hdd/global_user/xujiaming-253308120313/anaconda3/envs/specmod/lib/python3.14/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 5/5 [00:02<00:00,  1.73it/s]


In [11]:
print(tokenizer.decode(data[80]['input_ids']))

<|im_start|>system
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.<|im_end|>
<|im_start|>user
Darrell and Allen's ages are in the ratio of 7:11. If their total age now is 162, calculate Allen's age 10 years from now.<|im_end|>
<|im_start|>assistant
<think>
Okay, let's see. The problem says that Darrell and Allen's ages are in the ratio of 7:11, and their total age now is 162. I need to find Allen's age 10 years from now. Hmm, ratios can sometimes be tricky, but let me break it down step by step.

First, ratios. So, the ratio of their

In [ ]:
for meta_data in data:
    train_meta_data = {}
    train_meta_data['input_ids'] = meta_data['input_ids']
    print('1')
    train_meta_data['prompt_len'] = len(train_meta_data['input_ids']) - len(train_meta_data['layer_index'])

In [ ]:
len(data[2]['input_ids']) - len(data[2]['layer_index'])

In [ ]:
with open(f"/inspire/hdd/project/inference-chip/xujiaming-253308120313/Paper/SpecMoD/train_data/router/alpaca_Qwen3-8B_data_None_None.json", "r") as f:
    data = json.load(f)
    for id, json_data in data.items():
        print(len(json_data['output']))
        for idx, token_info in enumerate(json_data['Token']):
            print(token_info['layer_index'])
            break
        break

In [ ]:
print(data['1']['Token'][0])

In [ ]:
print(data[:,-2:-1,:])

In [ ]:
import torch
import torch.nn as nn
class ShadowAdapter2(nn.Module):
    def __init__(self, hidden_dim, bottleneck_dim=64):
        """
        Args:
            hidden_dim: 原模型 Hidden Size (e.g., Llama-3-8B is 4096)
            bottleneck_dim: 压缩后的维度 (e.g., 64 or 128), 越小越快
        """
        super().__init__()
        # 下投影：把维度压下去
        self.down_proj = nn.Linear(hidden_dim, bottleneck_dim, bias=False)
        # 激活函数：保持和 Llama 一致 (SiLU)
        self.act = nn.SiLU()
        # 上投影：把维度升回来
        self.up_proj = nn.Linear(bottleneck_dim, hidden_dim, bias=False)
        
        # 【关键技巧】零初始化 (Zero Initialization)
        # 让 up_proj 的权重一开始全是 0。
        # 这样初始时: Adapter(x) = 0, Output = x + 0 = x (完美透传)
        # 训练开始后，它会慢慢学到非 0 的修正值。
        
        nn.init.zeros_(self.up_proj.weight)
        
        # down_proj 正常随机初始化即可
        nn.init.kaiming_normal_(self.down_proj.weight, nonlinearity='linear')

    def forward(self, x):
        # 注意：这里我们只计算 Delta (残差部分)
        # 最终输出应该是 x + adapter(x)，在训练循环里加
        return self.up_proj(self.act(self.down_proj(x)))

In [ ]:

adaptor = nn.ModuleList([
            ShadowAdapter2(4096, 2048) for _ in range(36)
        ])
adaptor.load_state_dict(torch.load("/inspire/hdd/project/inference-chip/xujiaming-253308120313/Paper/SpecMoD/checkpoint/adaptor/2048_finetune/final_adapters.pt"))

In [ ]:
adaptor[0].down_proj.weight.dtype

In [ ]:
adaptor = adaptor.half()

In [ ]:
input = torch.rand((1,1,4096), dtype=torch.float16)
output = adaptor(input)
print(output.shape)

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM,Qwen3ForCausalLM
# /share/public/public_models/Qwen3-14B/
# /share/others/public_models/Qwen3-14B/

tokenizer = AutoTokenizer.from_pretrained("/inspire/hdd/global_public/public_models/Qwen/Qwen3-8B/")
model = AutoModelForCausalLM.from_pretrained("/inspire/hdd/global_public/public_models/Qwen/Qwen3-8B/")

In [ ]:
model.lm_head

In [ ]:
tokenizer.decode(2461)

In [ ]:
messages = [
            {"role": "system",
                "content": "You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information."},
        ]
prompt = 'How are you?'
messages.append({
    "role": "user",
    "content": prompt
})
inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    tokenize=True,
    return_dict=True,
    return_tensors="pt",
)

In [ ]:
inputs

In [ ]:
import json
nums = [0 for i in range(36)]
with open("/inspire/hdd/project/inference-chip/xujiaming-253308120313/Paper/SpecMoD/train_data/adaptor/mt-bench_Qwen3-8B_data_None_None.json", "r") as f:
    data = json.load(f)
for id, json_data in data.items():
    token = json_data['Token']
    for token_data in token:
        for idx in range(36):
            if idx not in token_data['layer_index']:
                nums[idx] += 1

In [ ]:
import torch
for i in range(36):
    data = torch.load(f'./train_data/adaptor/mt-bench_Qwen3-8B_X_idx{i}_None_None.pt')
    # print(data.shape[0] == nums[i], end='')
    print(data.shape)

In [3]:
data  = None
max(data, 0)

TypeError: '>' not supported between instances of 'int' and 'NoneType'

In [ ]:
len(data['1']['Token'])

In [ ]:
import torch
model_name = 'Qwen3-8B'
dataset = 'alpaca'
begin = 'None'
end = 'None'
cur_hidden_states = torch.load(f'./train_data/{dataset}_{model_name}_cur_hidden_states_{begin}_{end}.pt')
fake_last_hidden_states = torch.load(f'./train_data/{dataset}_{model_name}_fake_last_hidden_states_{begin}_{end}.pt')
true_last_hidden_states = torch.load(f'./train_data/{dataset}_{model_name}_true_last_hidden_states_{begin}_{end}.pt')

In [ ]:
spec_hidden_states = torch.load(f'./train_data/{dataset}_{model_name}_spec_hidden_states.pt')
label = torch.load(f'./train_data/{dataset}_{model_name}_label_data.pt')

In [ ]:
import torch
model_name = 'Qwen3-8B'
dataset = 'alpaca'
begin = 'None'
end = 'None'
train_data = torch.load(f'./train_data/{dataset}_{model_name}_train_data.pt')
label = torch.load(f'./train_data/{dataset}_{model_name}_label_data.pt')

In [ ]:
train_data.shape[0] // 80

In [ ]:
train_data = train_data.to(model.device)
for i in range(train_data.shape[0]):
    last_hidden_states = train_data[i,:,:train_data.shape[-1]//3]
    logits = model.lm_head(last_hidden_states)
    token = torch.argmax(logits, dim=-1)
    embeddings = model.model.embed_tokens(token)
    flag = embeddings == train_data[i,:,train_data.shape[-1]//3:train_data.shape[-1]//3 * 2]
    if not torch.all(flag):
        print(f'{i} Something wrong')

In [ ]:
last_hidden_states = train_data[1315,:,:train_data.shape[-1]//3]
logits = model.lm_head(last_hidden_states)
token = torch.argmax(logits, dim=-1)
embeddings = model.model.embed_tokens(token)

In [ ]:
embeddings

In [ ]:
train_data[1315,:,train_data.shape[-1]//3:train_data.shape[-1]//3 * 2]

In [ ]:
train_data = train_data[:,:,:train_data.shape[-1]//3*2]

In [ ]:
datasets = ['alpaca', 'gsm8k', 'math_infini', 'mt-bench', 'sum', 'vicuna-bench']
models = ['Qwen3-8B', 'Qwen3-14B']
dataset = 'alpaca'
model
spec_hidden_states_path = f'./train_data/{dataset}_{model}_spec_hidden_states.pt'
cur_hidden_states_path = f'./train_data/{dataset}_{model}_cur_hidden_states_None_None.pt'
last_hidden_states_path = f'./train_data/{dataset}_{model}_true_last_hidden_states_None_None.pt'
spec_hidden_states = torch.load(spec_hidden_states_path)
cur_hidden_states = torch.load(cur_hidden_states_path)
last_hidden_states = torch.load(last_hidden_states_path)

In [ ]:
models = ['Qwen3-8B', 'Qwen3-14B']
datasets = ['alpaca', 'gsm8k', 'sum', 'mt-bench','vicuna-bench', 'math_infini']
with open(f'')

In [ ]:
train_data.shape

In [ ]:
cur_hidden_states.shape

In [ ]:
fake_last_hidden_states.shape

In [ ]:
true_last_hidden_states.shape

In [ ]:
spec_hidden_states.shape

In [ ]:
label.shape

In [ ]:
a = torch.zeros(3,4)

In [ ]:
shape = a.shape

In [ ]:
import json
import torch
model_name = 'Qwen3-8B'
dataset = 'alpaca'
begin = 'None'
end = 'None'
with open(f"./train_data/{dataset}_{model_name}_data_{begin}_{end}.json", "r") as f:
    data = json.load(f)

In [ ]:
data['0']['Token'][1]

In [ ]:
datasets = ['alpaca', 'gsm8k', 'math_infini', 'mt-bench', 'sum', 'vicuna-bench']
models = ['Qwen3-8B', 'Qwen3-14B']
import os, json
for dataset in datasets:
    for model in models:
        train_data_path = f'./train_data/{dataset}_{model}_data_None_None.json'
        if os.path.exists(train_data_path):
            with open(train_data_path, 'r') as f:
                data = json.load(f)
                tot = 0
                for key, value in data.items():
                    tot += value['avg_len']
                print(f'{dataset}_{model}: {tot / len(data)}')

In [ ]:
tot = 0
for key, value in data.items():    
    tot += len(value['Token'])
print(tot)

In [ ]:
tmp = torch.tensor([[151667]])

In [ ]:
tmp.item() == data['0']['Token'][0]['input_id']

In [ ]:
output_id = 0
for key, value in data.items():
    for idx, token in enumerate(value['Token']):
        input_id = token['input_id']
        if  input_id != output_id:
            print(value['Token'][idx-1])
            print(idx, input_id, output_id)
        output_id = token['output_id']

In [ ]:
fake_logits = model.lm_head(fake_last_hidden_states[3])
true_logits = model.lm_head(true_last_hidden_states[4])
fake_token = torch.argmax(fake_logits, dim=-1)
true_token = torch.argmax(true_logits, dim=-1)
print(fake_token, true_token)

In [ ]:
print(model)

In [ ]:
messages = [
    {"role": "system",
        "content": "You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information."},
]
messages.append({
            "role": "user",
            "content": "How are you"
        })
inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    tokenize=True,
    return_dict=True,
    return_tensors="pt",
)

In [ ]:
inputs['input_ids'].shape

In [ ]:
import torch
min_dtype = torch.finfo(torch.float16).min
mask = torch.full((1, 117), fill_value=min_dtype, dtype=torch.float16)
cache_pos = torch.arange(117,118)
dag = torch.arange(117) > cache_pos.reshape(-1,1)

In [ ]:
dag

In [ ]:
import json

with open("/home/xujiaming/xujiaming/Paper/SpecMoD/data/alpaca_Qwen3-8B_data.json", "r") as f:
    data = json.load(f)
    lst = []
    for key, value in data.items():
        lst.append(value['avg_len'])
    print(sum(lst) / len(lst))

In [ ]:
lst = []
for key, value in data.items():
    lst.append(value['avg_len'])
print(sum(lst) / len(lst))

In [ ]:
with open("/home/xujiaming/xujiaming/Paper/SpecMoD/data/alpaca_Qwen3-8B_data.json", "r") as f:
    data = json.load(f)
    lst = []
    ground_truth = []
    for q_id, q_re in data.items():
        for token_id, token_value in q_re['Token'].items():
            lst.append(eval(token_id))
            ground_truth.append(token_value['layer_index'])

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
from model.qwen3_model import Spec_Qwen3ForCausalLM

model = Spec_Qwen3ForCausalLM.from_pretrained(f"/share/others/public_models/Qwen3-14B/")

In [ ]:
print(model.config)

In [ ]:
import torch
data_emb = model.model.embed_tokens(torch.tensor(lst)).detach()
print()

In [ ]:
data_emb.shape

In [ ]:
questions[0]

In [ ]:
a = [[[] for i in range(10)] for _ in range(10)]

In [ ]:
a={}

In [ ]:
a[100] = 10

In [ ]:
a

In [ ]:
a[0][0].append(5)

In [ ]:
a

In [ ]:
a[0][1] = a[0][0]

In [ ]:
import torch
import torch.nn.functional as F

v1 = torch.tensor([1., 2., 3.]).unsqueeze(0).unsqueeze(0)
v2 = torch.tensor([1., 1., 3.]).unsqueeze(0).unsqueeze(0)

# 对于 1D 向量，我们需要在 dim=0 上计算
similarity_1d = F.cosine_similarity(v1, v2, dim= -1).mean()
print(f"v1: {v1}")
print(f"v2: {v2}")
print(f"1D 向量余弦相似度: {similarity_1d}")

In [ ]:
import torch
a = torch.tensor([[1.,2.,3.]])

In [ ]:
torch.argmax(a).item()

In [ ]:
import torch.nn as nn
class PathPredictorMLP(nn.Module):
    def __init__(self, n_layers, llm_hidden_dim, mlp_internal_dim):
        super().__init__()
        
        self.input_dim = llm_hidden_dim 
        self.output_dim = n_layers
        
        self.net = nn.Sequential(
            nn.Linear(self.input_dim, mlp_internal_dim),
            nn.ReLU(),
            nn.LayerNorm(mlp_internal_dim), 
            nn.Linear(mlp_internal_dim, self.output_dim)
        )
    
    def forward(self, x):
        return self.net(x)
predictor = PathPredictorMLP(
    n_layers=36,
    llm_hidden_dim=4096,
    mlp_internal_dim=1024,)
predictor.load_state_dict(torch.load("/home/xujiaming/xujiaming/Paper/SpecMoD/path_predictor_mlp_baseline.pth"))


In [ ]:
predictor

In [ ]:
a = torch.randn(1, 1, 4096)
logits = predictor(a)

In [ ]:
logits > 0

In [ ]:
import torch
a = torch.arange(10)

In [ ]:
a[

In [ ]:
import torch
import torch.nn as nn

class ShadowAdapter(nn.Module):
    def __init__(self, hidden_dim, bottleneck_dim=64):
        """
        Args:
            hidden_dim: 原模型 Hidden Size (e.g., Llama-3-8B is 4096)
            bottleneck_dim: 压缩后的维度 (e.g., 64 or 128), 越小越快
        """
        super().__init__()
        # 下投影：把维度压下去
        self.down_proj = nn.Linear(hidden_dim, bottleneck_dim, bias=False)
        # 激活函数：保持和 Llama 一致 (SiLU)
        self.act = nn.SiLU()
        # 上投影：把维度升回来
        self.up_proj = nn.Linear(bottleneck_dim, hidden_dim, bias=False)
        
        # 【关键技巧】零初始化 (Zero Initialization)
        # 让 up_proj 的权重一开始全是 0。
        # 这样初始时: Adapter(x) = 0, Output = x + 0 = x (完美透传)
        # 训练开始后，它会慢慢学到非 0 的修正值。
        nn.init.zeros_(self.up_proj.weight)
        
        # down_proj 正常随机初始化即可
        nn.init.kaiming_normal_(self.down_proj.weight, nonlinearity='linear')

    def forward(self, x):
        # 注意：这里我们只计算 Delta (残差部分)
        # 最终输出应该是 x + adapter(x)，在训练循环里加
        return self.up_proj(self.act(self.down_proj(x)))

In [ ]:
adaptor = ShadowAdapter(4096, bottleneck_dim=64)

In [ ]:
state_dict = torch.load("/inspire/hdd/project/inference-chip/xujiaming-253308120313/Paper/SpecMoD/checkpoint/adaptor/adapter_layer_3.pt")
adaptor.load_state_dict(state_dict)

In [ ]:
adaptor

In [ ]:
adaptor.up_proj.weight